# Protein preparation - DNA / RNA implementation

### This is a generalised setup workflow adapted from KDPG / Cage examples for preparing a protein database (PDB) file for oxDNA simulation and / or visualisation with oxView, in conjunction with DNA preparation from a sequence file generated.

## Please place this notebook in the ANMUtils directory for best use.

## For ANMT simulation, follow ANMT Model headers.

### Please See Setup Notebook if you are experiencing dependency issues when executing the below command

In [ ]:
import models as m
import os
import subprocess

## Some useful directories, assumes directory structure of GitHub repo

current_folder = globals()['_dh'][0] # Get location of Jupyter Notebook .ipynb file
os.chdir(current_folder) # Make working directory the Jupyter Notebook .ipynb file location
print("Current working directory:", os.getcwd()) # Ensure this is the ANMUtils directory

bin_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/build/bin'
oxDNA_directory = bin_directory + '/oxDNA'
print("\nPredicted oxDNA path:",oxDNA_directory) # Double check this is the oxDNA directory

UTILS_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/UTILS'
print("\nPredicted UTILS path:",UTILS_directory) # Double check this is the UTILS directory

# Prep PDB File

### The scripts convert every CA atom in the provided PDB file including from repeated models. 

In [ ]:
#Set Working Directory // general directory work

wdir = os.getcwd()
print(wdir + '\n')
pdb_path = input("Enter the path of the .PDB file, relative to the working directory shown: \n")

parent_directory = os.path.abspath(os.path.join(pdb_path, os.pardir))

#Get name of protein from PDB filename 
protein_name = os.path.splitext(pdb_path)[0].split("/",-1)[-1]

# ANM Model

#### Generates .top, .dat and .par files from the .pdb file using relevant conversions.

In [ ]:
#ANM
# returns coordinates, b-factors, and chain map for EVERY CA in pdb file
coord, bfact, bmap = m.get_pdb_info(wdir + pdb_path, returntype='cbm')

# Initalize Model, T-> Temp in Kelvin that crystal was resolved at, cutoff-> r_max in Angstroms
anm = m.ANM(coord, bfact, T=300, cutoff=13)

# One shot Calculation of Analytical B-factors based of coordinates and cutoff value
anm.calc_ANM_unitary(cuda=False) # Only recommend enabling CUDA if (N > 1000)

# Compare Analytical to Experimental B-factors
anm.anm_compare_bfactors_jupyter(bmap=bmap)

# Pretty Good Fit to the Experimental B-factors, Lets export this network to simulation
m.export_to_simulation(anm, wdir + pdb_path, upstreamdir = parent_directory)

## ANM Model File manipulation

#### Changes the filenames to be suitable for the ANM oxDNA simulation

In [ ]:
# Moving Files From working Directory to Ex. Directory
os.chdir(wdir + parent_directory)
print(os.getcwd())

if os.path.isfile('generated.par') == True:
    os.replace('generated.par', str(protein_name)+'.par')
    os.replace('generated.top', str(protein_name)+'.top')
    os.replace('generated.dat', str(protein_name)+'.dat')
    print("Renamed files")
elif os.path.isfile(str(protein_name) + '.par') == True:
    print("Files already renamed")
else:
    print("No actionable files present")
    print(str(protein_name) + '.par')

current_folder = globals()['_dh'][0] # Get location of Jupyter Notebook .ipynb file
os.chdir(current_folder) # Make working directory the Jupyter Notebook .ipynb file location

# ANMT Model

#### Generates .top, .dat and .par files from the .pdb file using relevant conversions.

In [ ]:
# AMNT B-factors cannot yet be calculated analytically so we still use the ANM as our base

# There are two main differences. 1. in the input file we define kb kt (in sim units) and Interaction type = ACT
#                                 2. additional angle parameters in the parameter file

# returns coordinates separated by chain, and b-factors separated by chain
coord, bfact = m.get_pdb_info(wdir + pdb_path)

# To get the additional angle parameters we need to do the following
anmt = m.ANMT(coord, bfact, T=300, cutoff=20)

# One shot Calculation of Analytical B-factors based of coordinates and cutoff value (No knowledge of B&T)
anmt.calc_ANM_unitary(cuda=False)

# Export Simulation Files
m.export_to_simulation(anmt, wdir + pdb_path, upstreamdir = parent_directory)

## ANMT Model File manipulation

#### Changes the filenames to be suitable for the ANMT oxDNA simulation

In [ ]:
# Moving Files From working Directory to Ex. Directory
os.chdir(wdir + parent_directory)

if os.path.isfile('generated.par') == True:
    os.replace('generated.par', str(protein_name)+'t.par')
    os.replace('generated.top', str(protein_name)+'t.top')
    os.replace('generated.dat', str(protein_name)+'t.dat')
    print("Renamed files")
    
elif os.path.isfile(str(protein_name) + 't.par') == True:
    print("Files already renamed")
    
else:
    print("No actionable files present")
    print(str(protein_name) + 't.par')

current_folder = globals()['_dh'][0] # Get location of Jupyter Notebook .ipynb file
os.chdir(current_folder) # Make working directory the Jupyter Notebook .ipynb file location

# Prep DNA

Reads a text file with the following format:
- Each line contains the sequence for a single strand (A,C,T,G)
- Lines begining in DOUBLE produce double stranded DNA

Ex: Two ssDNA (single stranded DNA) <br>
ATATATA <br>
GCGCGCG

Ex: Two strands, one double stranded, the other single stranded. <br>
DOUBLE AGGGCT <br>
CCTGTA

## Create a short DNA sequence txt file below using the above formatting 

In [ ]:
current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

with open('sequence.txt', 'w') as writer:
    writer.close() # Reset / create sequence.txt

    
strands = int(input("How many strands are you inputting? \n"))
direction = int(input("Primary direction?\nType 3 for 3'-5' and 5 for 5'-3': \n"))

iters = 0
for iters in range(strands):

    sequ = str(input("Please type sequence {}: \n".format(iters + 1)))
    
    if direction == 5:
        
        if sequ[0:6] == "DOUBLE":
            sequ = "DOUBLE " + sequ[:6:-1]
            
        else:
            sequ = sequ[::-1]
            
        print("Converted to {} in 3'-5'\n ".format(sequ))
        
    with open('sequence.txt', 'a') as writer:
        writer.write(sequ)
        writer.write("\n")
    
    iters += 1
    
f = open('sequence.txt', 'r')
f.close()

print("File saved as sequence.txt in {}".format(current_folder))

## Convert sequence to necessary data and topology files 

In [ ]:
box_size = input("Enter desired box size:\n")
generate_sa = UTILS_directory + '/generate-sa.py'
generate_rna = UTILS_directory + '/generate-RNA.py'

if str(input("DNA or RNA\n")) == "DNA":
    subprocess.call(['python2', generate_sa, box_size, 'sequence.txt']) # Runs generate-sa.py in UTILS directory
else:
    subprocess.call(['python2', generate_rna, 'sequence.txt', box_size]) # Runs generate-RNA.py in UTILS directory
    
os.system(conf_directory  + ' input_AC')

# General input file generation

## Only have the .dat, .top and .par files wanted in the input file in the directory

### OXDNA2 = Standard DNA only model
### AC = ANM model
### ACT = ANMT model
### DNANM = DNA - ANM model
### DNACT = DNA - ANMT model

#### Produced is a general, formatted input file with the specifics needed for the simulation parameters chosen. Please edit the values within it as needed.

In [ ]:
current_folder = globals()['_dh'][0] # Get location of Jupyter Notebook .ipynb file
os.chdir(current_folder) # Make working directory the Jupyter Notebook .ipynb file location
sim_type = str(input("Type of simulation?\n oxDNA2 / AC / ACT / DNANM / DNACT \n")) # ACT = ANMT
int_type = str(input("MD or MC simulation? \n"))

subprocess.call(['python', "input_gen.py", sim_type, int_type]) # Runs input_gen.py in ANMUtils directory

## oxDNA2 Model Simulation

#### Ensure relevant input files are in the above printed directory along with the .top and .dat files. Alternatively, run the oxDNA executable in terminal using your preferred method on the relevant input file.

In [ ]:
# oxDNA executable path
current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

os.system(oxDNA_directory + ' input_oxDNA2')

## ANM Model Simulation

## To simulate protein and DNA together, please combine in oxView and create trap file first.

#### Ensure relevant input files are in the above printed directory along with the .par, .top and .dat files. Alternatively, run the oxDNA executable in terminal using your preferred method on the relevant input file.

In [ ]:
# ANM-oxDNA executable path
current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

os.system(oxDNA_directory + ' input_AC')

## ANMT Model Simulation

#### Ensure relevant input files are in the above printed directory along with the .par, .top and .dat files. Alternatively, run the oxDNA executable in terminal using your preferred method on the relevant input file.

## Protein relaxation:

In [ ]:
# ANMT-oxDNA executable path

current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

os.system(oxDNA_directory + ' input_ACT')

## Protein-DNA Interaction

In [ ]:
# ANM-oxDNA executable path

current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

os.system(oxDNA_directory + ' input_DNANM')

In [ ]:
# ANMT-oxDNA executable path

current_folder = globals()['_dh'][0]
os.chdir(current_folder)
print("Current working directory:", os.getcwd()) # Reset CWD

os.system(oxDNA_directory + ' input_DNACT')